# Create Custom Metric Provider for OpenScale
This notebook 
1. creates a deployment of how the metrics in a custom monitor need to be calculated, a.k.a. custom metric provider 
2. registers the custom metric provider in openscale
3. creates the corresponding monitor definition
4. updates monitor metadata

In [ ]:
import os
from cpd_sdk_plus import wml_sdk_utils as wml_util

In [ ]:
path_custom_metrics_script = 'custom_metrics_segmentation_edited.py'
conf = {'function_asset_name':'Custom Metrics Provider Function wendy',
             'function_deployment_name':'Custom Metrics Provider Deployment wendy',
             'openscale_integrated_system_name':"Custom Metrics Provider",
             'openscale_monitor_name':'Custom Metrics v999',
             'openscale_monitor_id':None,
             'openscale_monitor_defaults':
                {'metricA': {'threshold':[5,'lower']},
                 'metricB': {'threshold':[40,'lower']}}
             }

WML_SPACE_ID = '****'

WOS_GUID = '00000000-0000-0000-0000-000000000000'

In [ ]:
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)
wml_client.version

In [ ]:
wml_client.repository.list_functions()

## 1. Store and Deploy Custom Metrics Provider in the form of a Deployable Python function

The Python function receives the required variables, such as the datamart_id, monitor_instance_id, monitor_id, monitor_instance_parameters and subscription_id from the Watson OpenScale service when it is invoked by the custom monitor.

In the Python function, add your own logic to compute the custom metrics in the get_metrics method, publish the metrics to the Watson Openscale service and update the status of the run to the finished state in the custom monitor instance.

Update the WOS_CREDENTIALS in the Python function.

In [ ]:
function_asset_id = wml_util.function_store(path_custom_metrics_script,wml_client,
                                            function_name=conf['function_asset_name'],
                                            software_spec='runtime-22.1-py3.9')
print(function_asset_id)

In [ ]:
deployment_id,scoring_url = wml_util.function_deploy(function_asset_id,wml_client,
                                                      function_deployment_name=conf['function_deployment_name'])
print(deployment_id,scoring_url)

## 2. Register in OpenScale

In [ ]:
from ibm_watson_openscale import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *
from cpd_sdk_plus import wos_sdk_utils as wos_util

from datetime import datetime, timezone, timedelta
import uuid

In [ ]:
wos_client = wos_util.get_client()
wos_client.version

Update the custom metrics deployment URL, which is created during the Python function creation in the integrated system. Watson OpenScale invokes the deployment URL at runtime to compute the custom metrics.

You must define the authentication type based on the communication with custom metrics deployment. Watson OpenScale supports 2 types of authentication: basic and bearer. If custom metrics deployment accepts the basic authentication type, then provide auth_type=basic otherwise use auth_type=bearer.

In [ ]:
# Delete existing custom metrics provider integrated systems if present
wos_util.integrated_system_delete(conf['openscale_integrated_system_name'],wos_client)

In [ ]:
credentials = {}
with open(path_custom_metrics_script,'r') as f:
    for line in f:
        if "os.environ['USERNAME'] = " in line:
            credentials['username'] = eval(line.replace("os.environ['USERNAME'] = ",'').strip())
        elif "os.environ['APIKEY'] = " in line:
            credentials['api_key'] = eval(line.replace("os.environ['APIKEY'] = ",'').strip())
        else:
            pass

assert 'username' in credentials and 'api_key' in credentials, 'Either parsing has issue or the information is not included in the script'

In [ ]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=conf['openscale_integrated_system_name'],
    description=conf['openscale_integrated_system_name'],
    type="custom_metrics_provider",
    credentials= {"auth_type":"bearer",
                  "token_info": {
                      "url": "{}/icp4d-api/v1/authorize".format(os.environ['RUNTIME_ENV_APSX_URL']),
                      "headers": {"Content-Type": "application/json",
                                  "Accept": "application/json"},
                      "payload": {'username':credentials['username'],
                                   'api_key':credentials['api_key']},
                      "method": "post"}
                 },
    connection={"display_name": conf['openscale_integrated_system_name'],
                "endpoint": scoring_url
    }).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(integrated_system_id)

## 3. Setup custom monitor definition

In [ ]:
monitor_id = wos_util.monitor_definition_create(conf['openscale_monitor_name'],conf['openscale_monitor_defaults'],wos_client,overwrite=True)
print(monitor_id)

In [ ]:
wos_client.monitor_definitions.show()

## Update Metadata File

In [ ]:
metadata = {monitor_id:
            {'integrated_system_id':integrated_system_id,
             'wml_deployment_id':deployment_id}}

metadata

In [ ]:
# wml_util.metadata_yml_add(metadata,wml_client,metadata_type='monitor')
wml_util.metadata_yml_add(metadata,wml_client,metadata_type='monitor',overwrite=True)

In [ ]:
wml_util.metadata_yml_load(wml_client,metadata_type='monitor')